This is a simple Hello World example, showing how to take a function and run it with pywren. First we import the necessary libraries. 

In [11]:
import pywren
import numpy as np
import redis

Pywren is designed to run any existing python functions you have, in parallel, at scale, on the cloud. So first, we create an example python function. The funtion must take a single argument:

In [12]:
def my_function(x):
    return x + 7
my_function(3)

10

In [23]:
def my_function(x):
    client = redis.Redis(host="rediscluster.a9ith3.clustercfg.usw2.cache.amazonaws.com", port=6379)
    key = "foo"+str(x)
    client.set(key, str(x))
    r = client.get(key)
    if (r is None):
        print "no such key"
        return -1
    if r == str(x):
        print "success!"
    
    print r
    return r

To start using `pywren`, we first create an executor.

In [24]:
wrenexec = pywren.default_executor()

We can call `my_function(3)` remotely via `call_async`: 

In [25]:
future = wrenexec.call_async(my_function, 3)

Future is a placeholder for the returned value from applying `my_function` to the number `3`. We can call `result` on it and get the result. Note that this will block until the remote job has completed

In [26]:
future.result()

'3'

You can apply `my_function` to a list of arguments, and each will be executed remotely at the same time. 

In [27]:
futures = wrenexec.map(my_function, range(10))

The pywren `get_all_results` function will wait until all of the futures are done and return their results

In [28]:
pywren.get_all_results(futures)

ResponseError: MOVED 5173 172.31.19.131:6379